# [TPS-05] What if we assume that data are images ... Can we use convolution for this challange?

I love making AI fun. This is for fun ... or just this is not fun and you can find something cool with this experiment. I appreciate comments, feedback or any improvements for this notebook.

# 0. PREPARE

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
import umap
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras import backend as K
from keras.utils.vis_utils import plot_model
import matplotlib.pyplot as plt
import os
import tensorflow as tf

import warnings
warnings.filterwarnings("ignore")

In [ ]:
RANDOM_STATE = 2021
img_rows, img_cols = 5, 10

# 1. LOAD DATA AND TRANSFORM FOR NN

In [ ]:
train = pd.read_csv("../input/tabular-playground-series-may-2021/train.csv", index_col = 'id')
test = pd.read_csv("../input/tabular-playground-series-may-2021/test.csv", index_col = 'id')

In [ ]:
# Duplicates in dataset? This is noise ... kill them ....
# I find it thanks @omarvivas: https://www.kaggle.com/c/tabular-playground-series-may-2021/discussion/236561

train = train[~train.drop('target', axis = 1).duplicated()]
train.shape

In [ ]:
X = pd.DataFrame(train.drop("target", axis = 1))
lencoder = LabelEncoder()
y = pd.DataFrame(lencoder.fit_transform(train['target']), columns=['target'])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state= RANDOM_STATE)

In [ ]:
X_train = np.array(X_train, dtype= np.float32) 
X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
y_train = np.array(y_train)

X_test = np.array(X_test, dtype= np.float32) 
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
y_test = np.array(y_test)

test = np.array(test, dtype= np.float32)
test = test.reshape(test.shape[0], img_rows, img_cols, 1)

print(f'X_train shape: {X_train.shape}')
print(f'X_test shape: {y_train.shape}')
print(f'X_test shape: {X_test.shape}')
print(f'X_test shape: {y_test.shape}')

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
test = test.astype('float32')

X_train /= 255
X_test /= 255
test /= 255

# 2. VISUALIZE DATA (IMAGES)

## 2A. Show images

In [ ]:
num = 25
images = X_train[:num]
labels = y_train[:num]

num_row = 5
num_col = 5


fig, axes = plt.subplots(num_row, num_col, figsize=(3*num_col,4*num_row))
for i in range(num):
    ax = axes[i//num_col, i%num_col]
    ax.imshow(images[i], cmap='gray')
    ax.set_title('Label: {}'.format(labels[i]))
plt.tight_layout()
plt.show()

## 2B. TSNE

In [ ]:
# Lets look on TSNE  
train_sub = train.sample(10000, random_state= RANDOM_STATE)
model = TSNE(n_components=2, random_state=0, perplexity= 50, n_iter=3000)
tsne_data = model.fit_transform(StandardScaler().fit_transform(train_sub.drop('target', axis = 1).astype(float)))
tsne_data = np.vstack((tsne_data.T, train_sub.target)).T

tsne_df = pd.DataFrame(data=tsne_data, columns=("D1", "D2", "target"))

sns.FacetGrid(tsne_df, hue="target", height=6).map(plt.scatter, 'D1', 'D2').add_legend()
plt.title('Perplexity= 50, n_iter=3000')
plt.show()

## 2C. LDA

In [ ]:
train_sub = train.sample(10000, random_state= RANDOM_STATE)
lda_data = LDA(n_components=3).fit_transform(train_sub.drop(columns='target'),train_sub.target)
plt.figure(figsize=(10,10))
sns.scatterplot(x = lda_data[:, 0], y = lda_data[:, 1], hue = 'target', data=train_sub)

## 2D. UMAP

In [ ]:
train_sub = train.sample(10000, random_state= RANDOM_STATE)
embedding = umap.UMAP(random_state = RANDOM_STATE ,n_components=3).fit_transform(train_sub.drop(columns='target').to_numpy())
plt.figure(figsize=(10,10))
sns.scatterplot(x = embedding[:, 0], y = embedding[:, 1], hue='target', data=train_sub)

# 2. Neural Network (Keras 2DConv)
I take just simple NN architecture from cool MNIST dataset. You can just train this network as you can. ** Just play and have fun !!!

In [ ]:
batch_size = 128
num_classes = 4
epochs = 50

input_shape = (img_rows, img_cols, 1)

## 2A. CONVERT TO CATEGORICAL

In [ ]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

## 2B. DEFINE MODEL ARCHITECTURE

In [ ]:
# standard model ... nothing special ... 

model = Sequential()
x = Conv2D(256, kernel_size=(2, 2), padding='same', activation='relu', input_shape=input_shape)
model.add(x)
model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, (2, 2), padding='same', activation = 'relu'))
model.add(Conv2D(32, (2, 2), padding='same', activation = 'relu'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(63, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

## 2C. TRAIN 

In [ ]:
 earlystop = tf.keras.callbacks.EarlyStopping(
        monitor='val_accuracy',
        mode='max',
        patience=10, 
        verbose=1
    )

history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(X_test, y_test),
                    callbacks = earlystop)

## 2D. VISUALIZE TRAINING LOSS

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

## 2E. MODEL EVALUATE

In [ ]:
score = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy on test set: ",score[1])

## 2F. HACK THE MODEL

### A. FILTERS - first conv layer 

In [ ]:
filters, biases = x.get_weights()
conv_weight = filters[:,:,0,:]

# Check the shape of first Conv2D layer
print(f'First conv2D shape: {filters.shape}')
print(f'First conv2D output size: {x.output.shape} \n')

plt.figure(figsize = (20,10))
print("First 16 filters of conv2D layer")
for i in range(1,17):
    plt.subplot(4,4,i)
    plt.imshow(conv_weight[:,:,i], interpolation='nearest', cmap='gray', aspect='auto')

plt.show()

### B. OUTPUTS
How input looks like after filter application  ... 

In [ ]:
from numpy import expand_dims
from keras.models import Model

# I take one example from test dataset
img = expand_dims(X_test[0], axis=0)

# Then hijacked output from first layer
model_first2D = Model(inputs=model.inputs, outputs=x.output)

# Made prediction of first sample
feature_maps = model_first2D.predict(img)

# Plot all (32) images from our conv2D layer 
plt.figure(figsize = (40,20))
square = 8
ix = 1
for _ in range(4):
    for _ in range(square):
        ax = plt.subplot(square, square, ix)
        plt.imshow(feature_maps[0, :, :, ix-1], cmap='gray', interpolation='nearest')
        ix += 1
plt.show()

## 2F. PREDICT

In [ ]:
preds = model.predict_proba(test)

# 3. SUBMIT

In [ ]:
sub = pd.read_csv("/kaggle/input/tabular-playground-series-may-2021/sample_submission.csv")

predictions_df = pd.DataFrame(preds, columns = ["Class_1", "Class_2", "Class_3", "Class_4"])
predictions_df['id'] = sub['id']

In [ ]:
predictions_df.to_csv("conv_net_submission.csv", index = False)